In [1]:
import polars as pl
import numpy as np
from tqdm.auto import tqdm, trange
import glob
import altair as alt
import vegafusion as vf
vf.enable(row_limit=100000)
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [2]:
BASE_DIR = '/home/as8319/meng_project_traces/simpoints/spec2006/xalancbmk/0/input/m5out'

In [3]:
def build_dataset(path):
    # Read simpoint files
    with open(f'{path}/simpoint_weights.txt', 'r') as weights_file, \
        open(f'{path}/simpoints.txt', 'r') as intervals_file:

        lines = weights_file.readlines()
        
        intervals = np.empty(len(lines))
        weights   = np.empty(len(lines))
        
        for line in lines:
            weight, idx = line.split(' ')
            weights[int(idx)] = float(weight)

        lines = intervals_file.readlines()
        for line in lines:
            interval, idx = line.split(' ')
            intervals[int(idx)] = int(interval)
    
    # read dfs
    dfs = []
    for df_idx in trange(len(intervals)):
        dfs.append(pl.read_parquet(f'{path}/trace_{df_idx}.parquet'))
    
    # Sort by simpoint intervals
    # Traces are saved and labeled in order of their occurence
    # i.e. trace_0 corresponds to the lowest interval
    sort_idxs = np.argsort(intervals)
    for df_idx, simpoint_idx in enumerate(sort_idxs):
        dfs[df_idx] = dfs[df_idx].with_columns([
            pl.lit(weights[simpoint_idx]).alias('simpoint_weight'), 
            pl.lit(df_idx).alias('checkpoint')
        ])
    
    return pl.concat(dfs)

df = build_dataset(BASE_DIR)

  0%|          | 0/11 [00:00<?, ?it/s]

In [4]:
df.select(pl.col('tick').count())

tick
u32
23464214


In [5]:
df.head(3)

tick,disassembly,inst_addr,inst_rel_addr,pred_addr,pred_rel_addr,jump_addr,jump_rel_addr,pred_taken,mispredicted,ras,regs,virtual,return,call,taken,ras_rel,alloc_context,simpoint_weight,checkpoint
u64,str,u64,str,u64,str,u64,str,bool,bool,list[u64],struct[15],bool,bool,bool,bool,list[str],u64,f64,i32
95581813000,""" b.ne <_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+904>""",7281408,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+378""",7281412,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+37c""",7281412,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+37c""",false,false,"[5716856, 5736944, … 7040004]","{7829800,8021792,4294967295,10,7857352,8023800,19422108405268556,19422108405268556,104,8035696,62,31,8021810,8035680,46}",false,false,false,false,"[""_ZN10xalanc_1_829XalanSourceTreeContentHandler12startElementEPKtS2_S2_RKN11xercesc_2_510AttributesE+42c"", ""_ZN11xercesc_2_517SAX2XMLReaderImpl12startElementERKNS_14XMLElementDeclEjPKtRKNS_11RefVectorOfINS_7XMLAttrEEEjbb+7b8"", … ""_ZN11xercesc_2_512IGXMLScanner12scanDocumentERKNS_11InputSourceE+390""]",0,0.0015163,0
95581813000,""" cbnz x14, <_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+872>""",7281412,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+37c""",7281392,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+368""",7281392,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+368""",true,false,"[5716856, 5736944, … 7040004]","{7829800,8021792,4294967295,10,7857352,8023800,19422108405268556,19422108405268556,104,8035696,62,31,8021810,8035680,46}",false,false,false,true,"[""_ZN10xalanc_1_829XalanSourceTreeContentHandler12startElementEPKtS2_S2_RKN11xercesc_2_510AttributesE+42c"", ""_ZN11xercesc_2_517SAX2XMLReaderImpl12startElementERKNS_14XMLElementDeclEjPKtRKNS_11RefVectorOfINS_7XMLAttrEEEjbb+7b8"", … ""_ZN11xercesc_2_512IGXMLScanner12scanDocumentERKNS_11InputSourceE+390""]",0,0.0015163,0
95581814000,""" b.ne <_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+904>""",7281408,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+378""",7281412,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+37c""",7281412,"""_ZN10xalanc_1_818XalanDOMStringPool3getEPKtj+37c""",false,false,"[5716856, 5736944, … 7040004]","{7829800,8021792,4294967295,10,7857352,8023800,19422108405268556,19422108405268556,104,8035696,62,31,8021814,8035680,42}",false,false,false,false,"[""_ZN10xalanc_1_829XalanSourceTreeContentHandler12startElementEPKtS2_S2_RKN11xercesc_2_510AttributesE+42c"", ""_ZN11xercesc_2_517SAX2XMLReaderImpl12startElementERKNS_14XMLElementDeclEjPKtRKNS_11RefVectorOfINS_7XMLAttrEEEjbb+7b8"", … ""_ZN11xercesc_2_512IGXMLScanner12scanDocumentERKNS_11InputSourceE+390""]",0,0.0015163,0


In [6]:
df.lazy().groupby([
    pl.col('checkpoint'),
    pl.col('simpoint_weight')
]).agg([
    pl.col('mispredicted').cast(pl.UInt32).sum(),
    pl.count()
]).select([
    pl.col('mispredicted') * pl.col('simpoint_weight'),
    pl.col('count') * pl.col('simpoint_weight')
]).select(
    pl.col('mispredicted').sum() / pl.col('count').sum()
).collect(streaming=True)

mispredicted
f64
0.042561


In [38]:
ddf = df.lazy().groupby([
    pl.col('inst_addr'),
    pl.col('alloc_context'),
    pl.col('checkpoint'),
    pl.col('simpoint_weight'),
    pl.col('virtual'),
    pl.col('return')
]).agg(
    pl.col('taken').cast(pl.UInt32).sum(),
    pl.col('mispredicted').cast(pl.UInt32).sum(),
    pl.count()
).groupby([
    pl.col('inst_addr'),
    pl.col('alloc_context'),
    pl.col('virtual'),
    pl.col('return')
]).agg([
    (pl.col('taken') * pl.col('simpoint_weight')).sum(),
    (pl.col('mispredicted') * pl.col('simpoint_weight')).sum(),
    (pl.col('count') * pl.col('simpoint_weight')).sum()
]).with_columns([
    (pl.col('taken') / pl.col('count')).alias('taken_rate'),
    (pl.col('mispredicted') / pl.col('count')).alias('misprediction_rate'),
    pl.col('count').sum().alias('total count'),
    (pl.col('count') / pl.col('count').sum()).alias('share of occurences'),
    (pl.col('mispredicted') / pl.col('mispredicted').sum()).alias('share of mispredictions')
]).filter(
    (pl.col('alloc_context') != 0) # & pl.col('virtual') & ~pl.col('return')
).sort(
    'mispredicted',
    descending=True
).collect(streaming=True)

alt.Chart(ddf).mark_rect().encode(
    x=alt.X("taken_rate:Q", bin=True).axis(format='.0%'),
    y=alt.Y("misprediction_rate:Q", bin=True).axis(format='.0%'),
    color=alt.Color('sum(share of mispredictions):Q',
                    scale=alt.Scale(type='sqrt', scheme='blueorange',domain=[0, 0.1]),
                    legend=alt.Legend(format=".0%",title=['share of all', 'mispredictions']))
).properties(
    width=400,
    height=400
)

#ddf.head(5)

alt.Chart(...)

In [35]:
ddf = df.lazy().groupby([
    pl.col('inst_addr'),
    pl.col('checkpoint'),
    pl.col('simpoint_weight'),
]).agg(
    pl.col('taken').cast(pl.UInt32).sum(),
    pl.col('mispredicted').cast(pl.UInt32).sum(),
    pl.count()
).groupby([
    pl.col('inst_addr'),
]).agg([
    (pl.col('taken') * pl.col('simpoint_weight')).sum(),
    (pl.col('mispredicted') * pl.col('simpoint_weight')).sum(),
    (pl.col('count') * pl.col('simpoint_weight')).sum()
]).with_columns([
    (pl.col('taken') / pl.col('count')).alias('taken rate'),
    (pl.col('mispredicted') / pl.col('count')).alias('misprediction rate'),
    pl.col('count').sum().alias('total count'),
    (pl.col('count') / pl.col('count').sum()).alias('share of occurences'),
    (pl.col('mispredicted') / pl.col('mispredicted').sum()).alias('share of mispredictions')
]).collect(streaming=True)

p1 = alt.Chart(ddf).mark_rect().encode(
    x=alt.X("taken rate:Q", bin=True).axis(format='.0%'),
    y=alt.Y("misprediction rate:Q", bin=True).axis(format='.0%'),
    color=alt.Color('count():Q',
                    scale=alt.Scale(type='sqrt',scheme='blueorange'),
                    legend=alt.Legend(title=['#branches']))
).properties(
    width=400,
    height=400
)

p2 = alt.Chart(ddf).mark_rect().encode(
    x=alt.X("taken rate:Q", bin=True).axis(format='.0%'),
    y=alt.Y("misprediction rate:Q", bin=True).axis(format='.0%'),
    color=alt.Color('sum(share of mispredictions):Q',
                    scale=alt.Scale(type='sqrt', scheme='blueorange'),
                    legend=alt.Legend(format=".0%",title=['share of all', 'mispredictions']))
).properties(
    width=400,
    height=400
)

p3 = alt.Chart(ddf).mark_rect().encode(
    x=alt.X("taken rate:Q", bin=True).axis(format='.0%'),
    y=alt.Y("misprediction rate:Q", bin=True).axis(format='.0%'),
    color=alt.Color('sum(share of occurences):Q',
                    scale=alt.Scale(type='sqrt', scheme='blueorange'),
                    legend=alt.Legend(format=".0%",title=['share of all', 'branches']))
).properties(
    width=400,
    height=400
)

p3

alt.Chart(...)

In [39]:
df.lazy().groupby([
    pl.col('inst_addr'),
    pl.col('alloc_context'),
    pl.col('checkpoint'),
    pl.col('simpoint_weight'),
    pl.col('virtual'),
    pl.col('return')
]).agg(
    pl.col('taken').cast(pl.UInt32).sum(),
    pl.col('mispredicted').cast(pl.UInt32).sum(),
    pl.count()
).groupby([
    pl.col('inst_addr'),
    pl.col('alloc_context'),
    pl.col('virtual'),
    pl.col('return')
]).agg([
    (pl.col('taken') * pl.col('simpoint_weight')).sum(),
    (pl.col('mispredicted') * pl.col('simpoint_weight')).sum(),
    (pl.col('count') * pl.col('simpoint_weight')).sum()
]).with_columns([
    (pl.col('taken') / pl.col('count')).alias('taken rate'),
    (pl.col('mispredicted') / pl.col('count')).alias('misprediction rate'),
    pl.col('count').sum().alias('total count')
]).filter(
    (pl.min(pl.col('taken rate'), 1 - pl.col('taken rate')) < pl.col('misprediction rate')) & 
    (pl.col('alloc_context') != 0) # & pl.col('virtual') & ~pl.col('return')
).select([
    100 * ((pl.col('misprediction rate') - pl.min(pl.col('taken rate'), 1 - pl.col('taken rate'))) * pl.col('count') / pl.col('total count')).sum(),
]).collect(streaming=True)


literal
f64
2.440653
